## Imports

In [3]:
%pip install pylidc
%pip install spic
%pip install ipyvolume

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Global Vars


In [4]:
MIN_PID, MAX_PID = 1, 1012
MIN_HU, MAX_HU = -2048, 3071 # the actual min for HU is -1024, but we're looking for outside scan too
PLOT = 0 # plot while running, slows the process... a lot
NEW_SPACING = [1,1,1]

In [5]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
import plotly.figure_factory as FF
import pylidc as pl
import pydicom as dicom
import scipy
import skimage.data
import warnings

from ipywidgets import interact, widgets
from IPython.display import display
from scipy import ndimage as ndi
from skimage import measure, morphology
from skimage.color import rgb2gray
from skimage.filters import rank, roberts
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
#from scipy.ndimage import rotate
from skimage.transform import downscale_local_mean, rescale, rotate
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from pathlib import Path
from time import time

## Auxiliary functions


### Plots


In [6]:
def vol_histogram(volume, bins=50):
  plt.hist(volume.flatten(), bins=bins, color='c')
  plt.xlabel("Hounsfield Units (HU)")
  plt.ylabel("Frequency")
  plt.show()

In [7]:
def plot_ct_scan(scan, save=''):
    f, plots = plt.subplots(int(scan.shape[0] / 20) + 1, 4, figsize=(25, 25))
    for i in range(0, scan.shape[0], 5):
        plots[int(i / 20), int((i % 20) / 5)].axis('off')
        plots[int(i / 20), int((i % 20) / 5)].imshow(scan[i], cmap='gray')
    
    if save != '':
      plt.savefig(save)
      plt.close(f)
    else:
      plt.show() 

In [8]:
def sample_stack(stack, rows=7, cols=7, start_with=10, show_every=5, figsize=[25,25]):
    fig,ax = plt.subplots(rows,cols,figsize=figsize)
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

In [9]:
def slice_viewer(volume):
  def f(z):
      ax.imshow(volume[:,:,z], cmap='gray')
      fig.canvas.draw()
      display(fig)

  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)
  ax.imshow(volume[:,:,1], cmap="gray")

  interact(f, z=widgets.IntSlider(min=0,max=volume.shape[2]-1,step=1,value=round(volume.shape[2]))) 

In [10]:
def get_color(x,y,z):
  offset = 6
  x, y, z = int(x), int(y), int(z)
  x0, x1, y0, y1, z0, z1 = x-offset, x+offset, y-offset, y+offset, z-offset, z+offset
  #val = np.max(vol[x0:x1,y0:y1,z0:z1])
  val = int(np.mean(vol[x0:x1,y0:y1,z0:z1])) + 150
  return val

def make_mesh(image, threshold=-300, step_size=1):

    print("Transposing surface")
    p = image.transpose(0,1,2)
    
    print("Calculating surface")
    verts, faces= measure.marching_cubes_classic(p, threshold) 
    return verts, faces

def plotly_3d(vol, threshold=-300, step_size=1):

    verts, faces = make_mesh(vol, threshold, step_size)

    x,y,z = zip(*verts) 
    
    print("Drawing")
    
    # Make the colormap single color since the axes are positional not intensity. 
    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    #colormap=['rgba(26,150,65,0.5)','rgba(26,150,65,0.5)','rgba(26,150,65,0.5)']
    #colormap=['rgb(236, 236, 212)','rgb(236, 236, 212)']
    
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap='Greys',
                        color_func=get_color,
                        simplices=faces,
                        show_colorbar=False,
                        showbackground=False,
                        backgroundcolor='rgb(64, 64, 64)',
                        title="LIDC Example")
    
    #fig['data'][0].update(opacity=0.8) # transparency
    fig['layout']['scene']['xaxis']['visible'] = False
    fig['layout']['scene']['yaxis']['visible'] = False
    fig['layout']['scene']['zaxis']['visible'] = False
    
    iplot(fig)

In [11]:
# https://www.kaggle.com/tzeny15/full-preprocessing-tutorial

def plot_3d(image, threshold=-300, name='', savefile=''):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    #p = image.transpose(0,1,2)
    p = image
    
    verts, faces = measure.marching_cubes_classic(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    #face_color = [0.45, 0.45, 0.75]
    #mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    if name != '':
      ax.set_title(name)
    
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])
    if savefile != '':
      plt.savefig(savefile)
      print('Saved')
      plt.close(fig)
      mesh.remove()
    else:
      plt.show()

### Processing functions

In [12]:
# # https://www.kaggle.com/tzeny15/full-preprocessing-tutorial this was copied from below
# # https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial

# def largest_label_volume(im, bg=-1):
#     vals, counts = np.unique(im, return_counts=True)
#     #print(vals, counts)

#     counts = counts[vals != bg]
#     vals = vals[vals != bg]

#     if len(counts) > 0:
#         #print(vals[np.argmax(counts)])
#         return vals[np.argmax(counts)]
#     else:
#         return None

# def segment_lung_mask(image, fill_lung_structures=True, threshold=-320, plot=False, z_plot=None):
    
#     if z_plot is None:
#         z_plot = int(image.shape[2]/2)

#     #if plot:
#       #fig, ax = plt.subplots(1, 3, figsize=[30, 20])
      
#       #plotter(ax[0,0], image[:, :, z_plot], 'Normalized')
      


#     # not actually binary, but 1 and 2. 
#     # 0 is treated as background, which we do not want
#     binary_image = np.array(image > threshold, dtype=np.int8)+1
#     labels = measure.label(binary_image)
#     #labels[labels > 250] = 250
#     #print(labels)


#     if plot:
#       print('binary')
#       plt.imshow(binary_image[:, :, z_plot], cmap='gray')
#       plt.axis('off')
#       plt.show()
#       #plotter(ax[0], ((labels[:, :, z_plot]*10)^2)-1, 'a) Labeled')
#       #plotter(ax[0], binary_image[:, :, z_plot], 'a) Binary')
#       #plotter(ax[1], labels[:, :, z_plot], 'b) Air Filled', cmap='gray')

#     #print(2)
#     # Pick the pixel in the very corner to determine which label is air.
#     #   Improvement: Pick multiple background labels from around the patient
#     #   More resistant to "trays" on which the patient lays cutting the air 
#     #   around the person in half
#     background_label = labels[0,0,0]
#     #print(labels[0,0,0])
    
#     #print(3)
#     #Fill the air around the person
#     binary_image[background_label == labels] = 2
#     #binary_image[2 == labels] = 1

    
#     #if plot:
#       #plotter(ax[2], binary_image[:, :, z_plot], 'c) Filtered')


#     if plot:
#       print('air filled')
#       #plotter(ax[1], binary_image[:, :, z_plot])
#       plt.imshow(binary_image[:, :, z_plot], cmap='gray')
#       plt.axis('off')
#       plt.show()

#     # Method of filling the lung structures (that is superior to something like 
#     # morphological closing)
#     if fill_lung_structures:
#         # For every slice we determine the largest solid structure
#         for i, axial_slice in enumerate(binary_image):
#             axial_slice = axial_slice - 1
#             #if i == 64:
#               #plotter(ax[2], axial_slice, 'c) Filtered')
#             labeling = measure.label(axial_slice)
#             l_max = largest_label_volume(labeling, bg=0)
            
#             if l_max is not None: #This slice contains some lung
#                 #print(i)
#                 binary_image[i][labeling != l_max] = 1

#     #print(5)
#     #if plot:
#     #  plotter(ax[2], binary_image[:, :, z_plot], 'c) Filtered')

#     if plot:
#       print('filtered')
#       plt.imshow(binary_image[:, :, z_plot], cmap='gray')
#       plt.axis('off')
#       plt.show()

#     binary_image -= 1 #Make the image actual binary
#     binary_image = 1-binary_image # Invert it, lungs are now 1

#     #if plot:
#       #plotter(ax[2], binary_image[:, :, z_plot], 'c) True Binary')

#     #print(6)
#     # Remove other air pockets insided body
#     labels = measure.label(binary_image, background=0)
#     l_max = largest_label_volume(labels, bg=0)
#     if l_max is not None: # There are air pockets
#         binary_image[labels != l_max] = 0

#     #if plot:
#       #plotter(ax[1,2], binary_image[:, :, z_plot], 'Remove air Pockets')
#     #print(7)

#     #binary_image = fill_vol(binary_image)

#     #if plot:
#       #plotter(ax[3], binary_image[:, :, z_plot], 'd) Dilation')

#     #plt.show()

#     return binary_image

# #https://www.kaggle.com/ianyfchang/candidate-generation-and-luna16-preprocessing
# def fill_vol(volume):
#   def fill_slice(slice):
#     selem = disk(3)
#     slice = binary_closing(slice, selem)
#     #slice = roberts(slice)
#     #slice = ndi.binary_fill_holes(slice)
#     slice = ndi.binary_fill_holes(slice)
#     return slice

#   volume = np.asarray([fill_slice(s) for s in volume.transpose(2,0,1)]).transpose(1,2,0)
#   return volume

In [13]:
# def get_slope_intercept(id):
#   scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == id).first()
#   path = scan.get_path_to_dicom_files()
#   for s in os.listdir(path):
#     try:
#       slice = dicom.read_file(path + '/' + s, force=True)
#       intercept = slice.RescaleIntercept
#       slope = slice.RescaleSlope
#       return intercept, slope
#     except:
#       continue

# def get_pixels_hu(slices, intercept, slope):
#     #image = np.stack([s.pixel_array for s in slices])
#     # Convert to int16 (from sometimes int16), 
#     # should be possible as values should always be low enough (<32k)
#     image = vol.astype(np.int16)

#     # Set outside-of-scan pixels to 0
#     # The intercept is usually -1024, so air is approximately 0
#     image[image == -2000] = 0
    
#     # Convert to Hounsfield units (HU)
#     for slice_number in range(len(slices)):
        
#         #intercept = slices[slice_number].RescaleIntercept
#         #slope = slices[slice_number].RescaleSlope
        
#         if slope != 1:
#             image[slice_number] = slope * image[slice_number].astype(np.float64)
#             image[slice_number] = image[slice_number].astype(np.int16)
            
#         image[slice_number] += np.int16(intercept)
    
#     return np.array(image, dtype=np.int16)

In [14]:
# def normalizePlanes(npzarray, maxHU = 400., minHU = -1000.):
#     npzarray = (npzarray - minHU) / (maxHU - minHU)
#     npzarray[npzarray > 1] = 1
#     npzarray[npzarray < 0] = 0
#     return npzarray.astype('float32')

# def normalize(volume, min, max):
#     vol_std = (volume - volume.min()) / (volume.max() - volume.min())
#     vol_scaled = vol_std * (max - min) + min
#     return vol_scaled.astype('int16')

In [15]:
def mask_limits(volume, margin=5):

  x, y, z = volume.shape
  min_x, max_x = 0, x
  min_y, max_y = 0, y
  min_z, max_z = 0, z

  for i in range(x):
    s = volume[i, :, :]
    if s.sum() > 0.0:
      min_x = max( i - margin, 0)
      break

  for i in reversed(range(x-1)):
    s = volume[i, :, :]
    if s.sum() > 0.0:
      max_x = min( i + margin, x)
      break

  for i in range(y):
    s = volume[:, i, :]
    if s.sum() > 0.0:
      min_y = max( i - margin, 0)
      break

  for i in reversed(range(y-1)):
    s = volume[:, i, :]
    if s.sum() > 0.0:
      max_y = min( i + margin, y)
      break

  for i in range(z):
    s = volume[:, :, i]
    if s.sum() > 0.0:
      min_z = max( i - margin, 0)
      break

  for i in reversed(range(z-1)):
    s = volume[:, :, i]
    if s.sum() > 0.0:
      max_z = min( i + margin, z)
      break

  return min_x, max_x, min_y, max_y, min_z, max_z

def lung_limits(lungs, debug=False):
  z = lungs.shape[2]
  has_started = False
  has_ended = False
  min = 0
  max = 0
  for j in range(z):
    value = lungs[:,:,j].sum()
    if debug: 
      print(value)
    if not has_started:
      min = j - 2
    if value != 0:
      has_started = True
    if has_started and not has_ended:
      max = j + 2
    if value == 0 and has_started:
      has_ended = True
  
  if min < 0:
    min = 0
  if max == 0 or max > z:
    max = j - 1
  return min, max

In [16]:
# Pads volume to have desired shape
def to_shape(a, shape):
    y_, x_, z_ = shape
    y, x, z = a.shape
    y_pad = (y_-y)
    x_pad = (x_-x)
    z_pad = (z_-z)
    return np.pad(a,((y_pad//2, y_pad//2 + y_pad%2), 
                     (x_pad//2, x_pad//2 + x_pad%2),
                     (z_pad//2, z_pad//2 + z_pad%2)),
                  mode = 'constant')

In [17]:
def get_last_id(dir):
  last = MIN_PID
  
  if os.path.isdir(dir):
    try:
      paths = sorted(Path(dir).iterdir(), key=os.path.getmtime)
      last = int(str(paths[-1])[-8:-4])
    except Exception as e:
      print(e)
      last = MIN_PID
  else:
    os.mkdir(dir)

  print('Starting from scan', last)
    
  return last

In [18]:
def get_file_name(id):
  str_id = '000'+str(id) if id < 10 else '00'+str(id) if id < 100 else '0'+str(id) if id < 1000 else ''+str(id)
  return 'LIDC-IDRI-' + str_id + '.npy'

In [19]:
def get_spacing_resize_factor(id):
  scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == id).first()
  
  slice_thickness = scan.slice_thickness
  spacing = [scan.pixel_spacing] * 2 + [slice_thickness]
  spacing = np.array(spacing)

  resize_factor = spacing / NEW_SPACING

  return spacing, resize_factor

## Pre-Processing


### Normalize

In [20]:
# READ_DIR = '/gdrive/MyDrive/Data/LIDC/Numpy/'
# WRITE_DIR = '/gdrive/MyDrive/Data/LIDC_1/Numpy_Normalized/'

# last_pid = get_last_id(WRITE_DIR)

# total_time = 0
# nf = [] # ids not found or error
# for i in range(last_pid, MAX_PID + 1):
# #for i in range(1, 10):
#   start = time()

#   pid = get_file_name(i)
  
#   try:
#     vol = np.load(READ_DIR + pid)
#     if PLOT:
#       vol_histogram(vol,50)

#     vol = normalizePlanes(vol)
#     if PLOT:
#       vol_histogram(vol,50)

#     np.save(WRITE_DIR + pid, vol)
#   except Exception as e:
#     nf.append(pid)
#     print(e)

#   end = time()
#   total_time += round(end - start)
#   print(f'\rLast Scan: {i} || Time: {round(end-start)}s || Total time: {str(datetime.timedelta(seconds=total_time))} || Progress: {round((i / (MAX_PID + 1))*100,1)}%')

# print('\nVolumes not found:', nf)

In [21]:
# READ_DIR = '/gdrive/MyDrive/Data/LIDC_1/Numpy_Normalized/'

# total_time = 0
# nf = [] # ids not found or error
# avgs = []
# for i in range(1, MAX_PID + 1):
# #for i in range(1, 10):
#   start = time()

#   pid = get_file_name(i)
  
#   try:
#     vol = np.load(READ_DIR + pid)
    
#     avgs.append(vol.mean())

#     vol = normalizePlanes(vol)
#     if PLOT:
#       vol_histogram(vol,50)

#     np.save(WRITE_DIR + pid, vol)
#   except Exception as e:
#     nf.append(pid)
#     print(e)

#   end = time()
#   total_time += round(end - start)
#   print(f'\rLast Scan: {i} || Time: {round(end-start)}s || Total time: {str(datetime.timedelta(seconds=total_time))} || Progress: {round((i / (MAX_PID + 1))*100,1)}%')

# print('\nVolumes not found:', nf)

### Plotting

In [22]:
READ_DIR = '/gdrive/MyDrive/Data/LIDC_1/Numpy_Normalized_Masks/'
WRITE_DIR = '/gdrive/MyDrive/Data/LIDC_1/Numpy_Normalized_Masks_Plot/'
WRITE_FILE = '/gdrive/MyDrive/Data/LIDC_1/volumes.pkl'

last_pid = MIN_PID
volumes = {}
total_time = 0
nf = [] # ids not found or error

if os.path.isfile(WRITE_FILE):
  with open(WRITE_FILE, 'rb') as f:
    volumes = pkl.load(f)

last_pid = get_last_id(WRITE_DIR)


for i in range(1003, MAX_PID + 1):
  start = time()

  pid = get_file_name(i)
  
  try:

    print('Reading...', end='')
    mask = np.load(READ_DIR + pid)

    print('Plotting...', end='')
    plot_ct_scan(mask.transpose(2,0,1), WRITE_DIR + pid[:-4] + '.jpg')

  except Exception as e:
    nf.append(pid)
    print(e)
  
  end = time()
  total_time += round(end - start)
  print(f'Last Scan: {i} || Time: {round(end-start)}s || Total time: {str(datetime.timedelta(seconds=total_time))} || Progress: {round((i / (MAX_PID + 1))*100,1)}%\n')
  
print('Error on PID:', nf)

FileNotFoundError: [WinError 3] O sistema não conseguiu localizar o caminho especificado: '/gdrive/MyDrive/Data/LIDC_1/Numpy_Normalized_Masks_Plot/'